# Jupyter Notebook for Python

## A. Introduction

1. The default Runtime is "Python". Otherwise go to "Runtime" $\rightarrow$ "Change runtime type" and Choose "Python".
   
3. "Run" means
- Put the cursor on the cel and shift+enter on Jupyter Notebook
- Tap the circle near the upper left corner of the cell on Google Colab

## B. Run the following cell to read packages. 

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = False
import math
import numpy as np
import csv
import pprint
import pandas as pd
import requests
import io
import cv2
import pywt
import imageio.v2 as imageio
import requests
from io import BytesIO
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider
from ipywidgets import interact, IntSlider
from scipy.special import gammaln
from PIL import Image
from PIL import Image as PILImage 
from IPython.display import Image as IPyImage, display
from IPython.display import Image, display
from matplotlib.animation import FuncAnimation, PillowWriter

## Topics
1. The Collatz conjecture
2. RGB images
3. RGB movies
4. Singular value decomposition and low rank approximation
5. Wavelet decomposition
6. Public data and visualization
7. Central limit theorem for the binomial distribution $B(n,p)$
8. Taylor series of $\sin{x}$
9. Fourier series of a step function and a sawtooth function
10. Weyl's equidistribution theorem

## 1. The Collatz conjecture

Generate a sequence $\{x[n]\}_{n=1}^\infty$ by the following process:
1. Given an arbitrary positive integer $m$, set $x[1]:=m$. 
2. For $n=2,3,4,\dotsc$, if $x[n-1]:=1$, stop the process, otherwise 

\begin{aligned}
  x[n]
& :=\dfrac{x[n-1]}{2}\quad (x[n-1]\ \text{is even}),
\\
  x[n]
& :=
  3x[n-1]+1\quad (x[n-1]\ \text{is odd}).
\end{aligned}

In 1937 Lothar Collatz introduce the following conjecture:

#### For any $m=1,2,3,\dotsc$, there exists $n_m=1,2,3,\dotsc$ such that $x[n_m]=1$.

See [this paper](https://doi.org/10.1080/00029890.1985.11971528) for instance. Here are some examples: 

$1,
\quad 
2\rightarrow1, \quad 3\rightarrow10\rightarrow5\rightarrow16\rightarrow8\rightarrow4\rightarrow2\rightarrow1, 
\quad
4\rightarrow2\rightarrow1, 
\quad\dotsc$

Barina verified the Collatz conjecture is true for all $m\leqq2^{68}$ using workstations. See [this paper](https://doi.org/10.1007/s11227-020-03368-x). 

We examine this conjecture for $m=1,2,3,\dotsc,500$.

#### Run the following cell to create Collatz sequences for $x[1]=m=1,\dotsc,500$, and observe the behavior of each sequence. 

In [ ]:
# upper bound of m
M=500;

def collatz_sequence(m=1):
    x=np.ones(150, dtype=np.int64)
    N=np.ones(150, dtype=np.int64)
    for n in range(1,149):
        N[n]=n+1
        
    x[0]=int(m)
    for n in range(1,149):
        if x[n-1]==1:
            x[n]=x[n]
        elif x[n-1] % 2==0:
                    x[n]=int(x[n-1]/2)
        else: x[n]=3*x[n-1]+1

    fig, ax = plt.subplots(figsize=(7, 4))
    
    ax.scatter(N,x)
    ax.set_ylim(1, 1000)
    ax.set_title(f"Collatz sequence x[n] with x[1]={m}")
    ax.set_xlabel("n")
    ax.set_ylabel("x[n]")
    ax.set_xticks([0, 19, 39, 59, 79, 99, 119, 139], [1, 20, 40, 60, 80, 100, 120, 140])
    ax.grid(False)
    plt.show()

interact(collatz_sequence, m=(1,M,1));

#### Run the following cell to plot the pairs of $(m,n_m)$ for $m=1,\dotsc,500$ in a scatter graph.

In [ ]:
def collatz(n):
    return n//2 if n % 2 == 0 else 3*n+1

def stopping_time(m):
    steps = 0
    x = m
    while x > 1:
        x = collatz(x)
        steps += 1
    return steps

N = range(1, 501)
steps = [stopping_time(m) for m in N]
fig, ax = plt.subplots(figsize=(7, 4))
ax.scatter(N, steps, s=10)
ax.set_xlabel("Initial value m")
ax.set_ylabel("Steps n_m to 1")
ax.set_title("Collatz stopping step")
plt.show()

## 2. RGB images
The data of an RGB image is a triplet of matrices of the same size. This is essentially a triplet of grayscale images of the same size. These three grayscale images, colored red, green, and blue, are then overlaid to form an RGB image.
#### Run the following cell to create three $m{\times}n=10\times15$ matrices $R,G,B$ by random numbers. 

In [ ]:
# size of matrices
m, n = 10, 15 

Z = np.zeros((m,n))
R = np.random.rand(m,n)
G = np.random.rand(m,n)
B = np.random.rand(m,n)
R

#### Run the following cell to colorize  $R,G,B$, and show red, green, blue and RGB images.

In [ ]:
imgR = np.dstack([R, Z, Z])
imgG = np.dstack([Z, G, Z])
imgB = np.dstack([Z, Z, B])
imgRGB = np.dstack([R, G, B])

# 2×2 
fig, axs = plt.subplots(2, 2, figsize=(6,5))

axs[0,0].imshow(imgR)
axs[0,0].set_title("Grayscale Image (Red)")
axs[0,0].axis("off")

axs[0,1].imshow(imgG)
axs[0,1].set_title("Grayscale Image (Green)")
axs[0,1].axis("off")

axs[1,0].imshow(imgB)
axs[1,0].set_title("Grayscale Image (Blue)")
axs[1,0].axis("off")

axs[1,1].imshow(imgRGB)
axs[1,1].set_title("RGB Image")
axs[1,1].axis("off")

plt.tight_layout()
plt.show()

## 3. RGB movies
The data of an RGB movie is the finite number of triplets of matrices of the same size. This is essentially a sequence of RGB images of the same size. 
#### Run the following cell to create 100 RGB images by random numbers and see the movie.

In [ ]:
# size of matrices
m, n = 10, 15 
# length of the movie
l=100

rng = np.random.default_rng(0)
R = rng.integers(80, 256, size=(m, n, l)) / 255.0
G = rng.integers(80, 256, size=(m, n, l)) / 255.0
B = rng.integers(80, 256, size=(m, n, l)) / 255.0
Z = np.zeros((m, n), dtype=float)

def frame_images(k):
    imgR   = np.dstack([R[:, :, k], Z, Z])
    imgG   = np.dstack([Z, G[:, :, k], Z])
    imgB   = np.dstack([Z, Z, B[:, :, k]])
    imgRGB = np.dstack([R[:, :, k], G[:, :, k], B[:, :, k]])
    return imgR, imgG, imgB, imgRGB

fig, axs = plt.subplots(2, 2, figsize=(6, 5))
titles = ["Grayscale (Red)", "Grayscale (Green)", "Grayscale (Blue)", "RGB Image"]
for ax, t in zip(axs.ravel(), titles):
    ax.set_title(t)
    ax.axis("off")

imgR0, imgG0, imgB0, imgRGB0 = frame_images(0)
artists = [
    axs[0,0].imshow(imgR0,   vmin=0, vmax=1),
    axs[0,1].imshow(imgG0,   vmin=0, vmax=1),
    axs[1,0].imshow(imgB0,   vmin=0, vmax=1),
    axs[1,1].imshow(imgRGB0, vmin=0, vmax=1),
]

def update(k):
    imgR, imgG, imgB, imgRGB = frame_images(k)
    artists[0].set_data(imgR)
    artists[1].set_data(imgG)
    artists[2].set_data(imgB)
    artists[3].set_data(imgRGB)
    return artists

anim = FuncAnimation(fig, update, frames=l, interval=333, blit=False)
anim.save("rgb_animation.gif", writer=PillowWriter(fps=3))
plt.close(fig)

display(Image(filename="rgb_animation.gif"))

## 4. Singular value decomposition and low rank approximation
In general an $m{\times}n$ matrix $A=\begin{bmatrix}\vec{a}_1 & \dotsb & \vec{a}_n \end{bmatrix}$ has a specific nonnegative integer which is called as the rank of  $A$ defined by 

$$r
:=
\operatorname{dim}\bigl(
\{
A\vec{x}
=
x_1\vec{a}_1+\dotsb+x_n\vec{a}_n 
\ \vert \ 
\vec{x} \in \mathbb{R}^n
\}
\bigr).$$ 

It follows that $1 \leqq r \leqq \min\{m,n\}$ unless all the entries of $A$ are zero. It is known that there exist  

$$\sigma_1 \geqq \dotsb \geqq \sigma_r>0,
\quad
\vec{u}_1,\dotsc,\vec{u}_r \in \mathbb{R}^m,
\quad
\vec{v}_1,\dotsc,\vec{v}_r \in \mathbb{R}^n$$

such that we have the **singular value decomposition**:

$$A=\sum_{j=1}^r\sigma_j\vec{u}_j\vec{v}_j^T.$$

Set 

$$A_k:=\sum_{j=1}^k\sigma_j\vec{u}_j\vec{v}_j^T, \quad k=1,\dotsc,r.$$

It is known that the rank of $A_k$ is $k$ and closest to $A$ among all the $m{\times}n$ matrices of rank $k$, that is, $A_k$ is the best approximation of $A$ among all the $m{\times}n$ matrices of rank $k$.   

We compute a $432\times768\times3$ RGB image of a hibiscus. All the R, G, B matrices are of rank $432$.

#### Run the following cell to see the movie of the best low rank approximation for $r=1,\dotsc,60$.

In [ ]:
url = "https://raw.githubusercontent.com/fiomfd/hands-on/refs/heads/main/data/hibiscus.jpg"
response = requests.get(url)
I = PILImage.open(BytesIO(response.content))

new_size = (I.width // 5, I.height // 5)
X = I.resize(new_size, resample=PILImage.BICUBIC)
X = np.asarray(X, dtype=float) / 255.0  
p, q = X.shape[:2]
R = X[:, :, 0]
G = X[:, :, 1]
B = X[:, :, 2]

def svd3(A):
    U, S, VT = np.linalg.svd(A, full_matrices=False)   # A ≈ U @ diag(S) @ VT
    return U, S, VT

RU, RS, RVT = svd3(R)
GU, GS, GVT = svd3(G)
BU, BS, BVT = svd3(B)

rank = 100
rank = min(rank, min(p, q)) 

DR = np.zeros((p, q, rank), dtype=float)
DG = np.zeros((p, q, rank), dtype=float)
DB = np.zeros((p, q, rank), dtype=float)

for k in range(1, rank + 1):
    DR[:, :, k-1] = (RU[:, :k] * RS[:k]) @ RVT[:k, :]
    DG[:, :, k-1] = (GU[:, :k] * GS[:k]) @ GVT[:k, :]
    DB[:, :, k-1] = (BU[:, :k] * BS[:k]) @ BVT[:k, :]

p, q = R.shape
rank = DR.shape[2]

T = rank + 16                       
Y = np.zeros((T, p, q, 3), float)   

Y[0:4, :, :, 0] = R
Y[0:4, :, :, 1] = G
Y[0:4, :, :, 2] = B

Y[8:8+rank, :, :, 0] = np.transpose(DR, (2, 0, 1)) 
Y[8:8+rank, :, :, 1] = np.transpose(DG, (2, 0, 1))
Y[8:8+rank, :, :, 2] = np.transpose(DB, (2, 0, 1))

Y[rank+12:rank+16, :, :, 0] = R
Y[rank+12:rank+16, :, :, 1] = G
Y[rank+12:rank+16, :, :, 2] = B

W = np.stack([
    np.transpose(DR, (2, 0, 1)),  # (rank, p, q)
    np.transpose(DG, (2, 0, 1)),
    np.transpose(DB, (2, 0, 1))
], axis=-1)  # -> (rank, p, q, 3)

def show_rank(r):
    approx = np.clip(W[r-1], 0, 1)  # そのまま (p, q, 3)
    fig, axes = plt.subplots(1, 2, figsize=(8,4))
    axes[0].imshow(approx, vmin=0, vmax=1)
    axes[0].set_title(f"approximation rank = {r}")
    axes[0].axis("off")

    axes[1].imshow(np.clip(X, 0, 1), vmin=0, vmax=1)
    axes[1].set_title("original RGB image")
    axes[1].axis("off")
    plt.tight_layout(); plt.show()

interact(show_rank, r=IntSlider(min=1, max=W.shape[0], step=1, value=1, description="rank"));

## 5. Wavelet decomposition

The discrete wavelet is a pair of orthonomal two vectors $\vec{u}$ and $\vec{v}$ satisfying some condition, and the most typical example of the discrete wavelets is the Haar wavelet. They are very simple vectors: 

$$\vec{u}
=
\frac{1}{\sqrt{2}}
\begin{bmatrix}
1
\\
1
\\
0
\\
\vdots
\\
0 
\end{bmatrix}, 
\quad
\vec{v}
=
\frac{1}{\sqrt{2}}
\begin{bmatrix}
1
\\
-1
\\
0
\\
\vdots
\\
0 
\end{bmatrix}
\in\mathbb{R}^{N},$$

Roughly speaking the discrete wavelet transform by the Haar wavelet is to take averages of neighboring elements. The elements of a vector are transformed  to the average of neighboring two elements, and the elements of a matrix are converted to the average of neighboring 2 by 2 elements. The following examples show the outcomes of repeating the discere wavelet tranform twice: 

$$\vec{a}_0
=
\begin{bmatrix}
1 \\ 3 \\ 5 \\ 7 
\end{bmatrix}
\mapsto 
\vec{a}_1
=
\begin{bmatrix}
2 \\ 2 \\ 6 \\ 6 
\end{bmatrix}
\mapsto 
\vec{a}_2
=
\begin{bmatrix}
4 \\ 4 \\ 4 \\ 4 
\end{bmatrix}$$

$$A_0
=
\begin{bmatrix}
0 & 2 & 4 & 6
\\ 
8 & 10 & 12 & 14 
\\ 
16 & 18 & 20 & 22
\\ 
24 & 26 & 28 & 30
\end{bmatrix}
\mapsto 
A_1
=
\begin{bmatrix}
5 & 5 & 9 & 9
\\ 
5 & 5 & 9 & 9 
\\ 
21 & 21 & 25 & 25
\\ 
21 & 21 & 25 & 25
\end{bmatrix}
\mapsto 
A_2
=
\begin{bmatrix}
15 & 15 & 15 & 15
\\ 
15 & 15 & 15 & 15 
\\ 
15 & 15 & 15 & 15
\\ 
15 & 15 & 15 & 15
\end{bmatrix}.$$


The output such as $\vec{a}_\ell$ and $A_\ell$ of $\ell$ times operations is said to be the approximation part of level $\ell$, and the remainder term such as $\vec{a}_0-\vec{a}_\ell$ and $A_0-A_\ell$ is called the detail part of level $\ell$. In terms of Fourier analysis, the approximation part is the low frequency part, and the detail part is the high frequency part and is similar to the edge detection. 

We observe the wavelet decomposition of the $384\times512$ RGB image of fried noodle using the Haar wavelet. We can get the wavelet decomposition up to level $7$ since $384=2^7\times3$ and $512=2^9$. 

#### Run the following cell to implement the wavelet decomposition.

In [ ]:
url2 = "https://raw.githubusercontent.com/fiomfd/hands-on/refs/heads/main/data/CityU.jpg"
response = requests.get(url2)
I8 = PILImage.open(BytesIO(response.content))

new_size = (I8.width // 8, I8.height // 8)
X8 = I8.resize(new_size, resample=PILImage.BICUBIC)
X8 = np.asarray(X8, dtype=float) / 255.0  
p8, q8 = X8.shape[:2]
R8 = X8[:, :, 0]
G8 = X8[:, :, 1]
B8 = X8[:, :, 2]

L = 7
wave = 'haar'
mode = 'periodization'  

A8 = np.stack([R8, G8, B8], axis=0)

XR = np.zeros((p8, q8, L), float)
XG = np.zeros((p8, q8, L), float)
XB = np.zeros((p8, q8, L), float)
R_slices, G_slices, B_slices = [], [], []  

for l in range(1, L+1):
    Rc = pywt.wavedec2(R8, wave, level=l, mode=mode)
    Gc = pywt.wavedec2(G8, wave, level=l, mode=mode)
    Bc = pywt.wavedec2(B8, wave, level=l, mode=mode)

    Rarr, Rsl = pywt.coeffs_to_array(Rc)
    Garr, Gsl = pywt.coeffs_to_array(Gc)
    Barr, Bsl = pywt.coeffs_to_array(Bc)

    XR[:, :, l-1] = Rarr
    XG[:, :, l-1] = Garr
    XB[:, :, l-1] = Barr
    R_slices.append(Rsl); G_slices.append(Gsl); B_slices.append(Bsl)

XRapprox = np.zeros_like(XR)
XGapprox = np.zeros_like(XG)
XBapprox = np.zeros_like(XB)

for l in range(1, L+1):
    XRapprox[:, :, l-1][R_slices[l-1][0]] = XR[:, :, l-1][R_slices[l-1][0]]
    XGapprox[:, :, l-1][G_slices[l-1][0]] = XG[:, :, l-1][G_slices[l-1][0]]
    XBapprox[:, :, l-1][B_slices[l-1][0]] = XB[:, :, l-1][B_slices[l-1][0]]

XRdetail = XR - XRapprox
XGdetail = XG - XGapprox
XBdetail = XB - XBapprox

def rec_from_array(arr2d, slices):
    coeffs = pywt.array_to_coeffs(arr2d, slices, output_format='wavedec2')
    rec = pywt.waverec2(coeffs, wave, mode=mode)
    return rec[:p8, :q8]  

YRapprox = np.zeros((p8, q8, L), float)
YGapprox = np.zeros((p8, q8, L), float)
YBapprox = np.zeros((p8, q8, L), float)
YRdetail = np.zeros((p8, q8, L), float)
YGdetail = np.zeros((p8, q8, L), float)
YBdetail = np.zeros((p8, q8, L), float)

for l in range(1, L+1):
    YRapprox[:, :, l-1] = rec_from_array(XRapprox[:, :, l-1], R_slices[l-1])
    YGapprox[:, :, l-1] = rec_from_array(XGapprox[:, :, l-1], G_slices[l-1])
    YBapprox[:, :, l-1] = rec_from_array(XBapprox[:, :, l-1], B_slices[l-1])

    YRdetail[:, :, l-1] = rec_from_array(XRdetail[:, :, l-1], R_slices[l-1])
    YGdetail[:, :, l-1] = rec_from_array(XGdetail[:, :, l-1], G_slices[l-1])
    YBdetail[:, :, l-1] = rec_from_array(XBdetail[:, :, l-1], B_slices[l-1])

Wapprox = np.zeros((3, p8, q8, L+1), float)
Wdetail = np.zeros((3, p8, q8, L+1), float)

Wapprox[:, :, :, 0] = A8  
for l in range(1, L+1):
    Wapprox[0, :, :, l] = YRapprox[:, :, l-1]
    Wapprox[1, :, :, l] = YGapprox[:, :, l-1]
    Wapprox[2, :, :, l] = YBapprox[:, :, l-1]

    Wdetail[0, :, :, l] = YRdetail[:, :, l-1]
    Wdetail[1, :, :, l] = YGdetail[:, :, l-1]
    Wdetail[2, :, :, l] = YBdetail[:, :, l-1]

Wapprox = np.clip(Wapprox, 0, 1)
Wdetail = np.clip(Wdetail, 0, 1)

def show_level(l):
    fig, axes = plt.subplots(1, 3, figsize=(12, 3))
    for ax in axes: ax.axis('off')

    axes[0].imshow(np.moveaxis(A8, 0, -1), vmin=0, vmax=1)               # Original
    axes[0].set_title("Original RGB",fontsize=16)

    axes[1].imshow(np.moveaxis(Wapprox[:, :, :, l], 0, -1), vmin=0, vmax=1)
    axes[1].set_title(f"Approximation (level {l})",fontsize=16)

    axes[2].imshow(np.moveaxis(Wdetail[:, :, :, l], 0, -1), vmin=0, vmax=1)
    axes[2].set_title(f"Detail (level {l})",fontsize=16)

    plt.tight_layout(); plt.show()

interact(show_level, l=IntSlider(min=0, max=L, step=1, value=0, description="Level"));

## 6. Public data and visualization

Let $\vec{x}=\bigl[x[1],\dotsc,x[N]\bigl]^T\in\mathbb{R}^N$ be an $N$-dimensional real column vector. This can describes data such as $x[i]$ means the highest temperature of the $i$-th day for $i=1,\dotsc,N$. One can exploit some information such as the average $m$ and the variance $\sigma^2$:

$$m:=\frac{1}{N}\sum_{i=1}^Nx[i],
\quad
\sigma^2:=\frac{1}{N}\sum_{i=1}^N(x[i]-m)^2.$$

In what follows Julia downloads open data, and visualizes information taking averages, maximums and minimums. More concretely Julia downloads 
- the daily maximum tenperature
- the daily minimum tenpereture
- the daily mean tenperature 
of 1884-present at the Hong Kong Observatory. The starting dates of  the theree data are not same, and so we make use of data of 140 years of 1885-2024. The Hong Kong Observatory dataset has no daily temperature records from 1 Jan 1940 to 31 Dec 1946, due to the disruption of WWII and the Japanese occupation. This 7-year gap means that analyses requiring continuous daily data over multiple years will be affected if these years are included. To ensure consistency, we restrict the analysis period to 1 Jan 1947 – 31 Dec 2024.

#### Run the following cell to donwload the public data and show the table.

In [ ]:
urlmax = 'https://data.weather.gov.hk/weatherAPI/opendata/opendata.php?dataType=CLMMAXT&rformat=csv&station=HKO'
namemax = 'HK_Maximun_Temperature.csv'

r = requests.get(urlmax)
r.raise_for_status()   
with open(namemax, "wb") as f:
    f.write(r.content)
Amax = pd.read_csv(namemax, encoding="utf-8-sig", sep=None, skiprows=2, skipfooter=3, usecols=[0, 1, 2, 3], engine="python")

Amax.rename(columns={
    "年/Year": "Year",
    "月/Month": "Month",
    "日/Day": "Day",
    "數值/Value": "Maximum",
    "數據完整性/data Completeness": "Completeness"
}, inplace=True)

urlmean = 'https://data.weather.gov.hk/weatherAPI/opendata/opendata.php?dataType=CLMTEMP&rformat=csv&station=HKO'
namemean = 'HK_Mean_Temperature.csv'

r = requests.get(urlmean)
r.raise_for_status()   
with open(namemean, "wb") as f:
    f.write(r.content)
Amean = pd.read_csv(namemean, encoding="utf-8-sig", sep=None, skiprows=2, skipfooter=3, usecols=[0,1,2,3], engine="python")

Amean.rename(columns={
    "年/Year": "Year",
    "月/Month": "Month",
    "日/Day": "Day",
    "數值/Value": "Average",
    "數據完整性/data Completeness": "Completeness"
}, inplace=True)

urlmin = 'https://data.weather.gov.hk/weatherAPI/opendata/opendata.php?dataType=CLMMINT&rformat=csv&station=HKO'
namemin = 'HK_Minimun_Temperature.csv'

r = requests.get(urlmin)
r.raise_for_status()   
with open(namemin, "wb") as f:
    f.write(r.content)
Amin = pd.read_csv(namemin, encoding="utf-8-sig", sep=None, skiprows=2, skipfooter=3, usecols=[0,1,2,3], engine="python")

Amin.rename(columns={
    "年/Year": "Year",
    "月/Month": "Month",
    "日/Day": "Day",
    "數值/Value": "Minimum",
    "數據完整性/data Completeness": "Completeness"
}, inplace=True)

Amax_1885 = Amax[Amax["Year"] >= 1885]
Amean_1885 = Amean[Amean["Year"] >= 1885]
Amin_1885  = Amin[Amin["Year"] >= 1885]
A_1885 = Amax_1885.merge(Amean_1885[["Year", "Month", "Day", "Average"]], on=["Year", "Month", "Day"])
A_1885 = A_1885.merge(Amin_1885[["Year", "Month", "Day", "Minimum"]], on=["Year", "Month", "Day"])
A_1947_2024 = A_1885[(A_1885["Year"] >= 1947) & (A_1885["Year"] <= 2024)]
A_1947_2024 = A_1947_2024.copy()

print(A_1885);

#### Run the following cell to draw the line graphs of Annual maximum, average, and minimum temperatures.
Over the long term, temperatures are rising slowly.

In [ ]:
numcols = ["Maximum", "Average", "Minimum"]
for c in numcols:
    A_1947_2024[c] = (
        A_1947_2024[c]
        .astype(str)
        .str.replace(r"[^\d\.\-]", "", regex=True) 
        .replace({"": None})                       
        .pipe(pd.to_numeric, errors="coerce")      
    );

Y = A_1947_2024.groupby("Year").agg(
    Ymax=("Maximum", "max"),
    Yave=("Average", "mean"),
    Ymin=("Minimum", "min")
)

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(Y)
ax.set_title("Annual max, ave, and min temperatures at HK Observatory", fontsize=10);
ax.set_xlim([1940,2030]);
ax.set_ylim([0,40]);
ax.set_xlabel("year");
ax.set_ylabel("C");
plt.show()

#### Run the following cell to create a movie showing the monthy maximum, average, and minmum temperatures in each year between 1947 and 2024.

In [ ]:
M = A_1947_2024.groupby(["Year","Month"]).agg(
    Mmax=("Maximum", "max"),
    Mave=("Average", "mean"),
    Mmin=("Minimum", "min")
);

def hktemp(year=1947):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_title(f"Monthly max, ave, and min temperatures at HK Observatory in {year}", fontsize=9)
    ax.plot(M.loc[year])
    ax.set_ylim([0,40])
    ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
    ax.set_yticks([0,5,10,15,20,25,30,35,40])
    ax.set_xlabel("month");
    ax.set_ylabel("C");
    ax.grid(False)
    plt.show()
interact(hktemp,year=(1947, 2024));

## 7. Central limit theorem for the binomial distribution $B(n,p)$
Let $n$ be a positive integer, and let $p$ be a constant satisfying $0 < p < 1$. 
Denote by $B(n,p)$ the the binomial distribution of $n$ iid experiments $X_1,\dotsc,X_n$ with probability $p$, that is, a discrete probability distribution on $\{0,1,\dotsc,n\}$ such that 

$$\operatorname{P}(S_n=k)=\frac{n!}{k!(n-k)!}p^k(1-p)^{n-k}, 
\quad
k=0,1,\dotsc,n,$$

where $S_n:=X_1+\dotsb+X_n$.

It is well-known that for any $a,b\in\mathbb{R}$ with $a<b$, 

$$\operatorname{P}
\left(
a \leqq \frac{S_n-pn}{\sqrt{np(1-p)}} \leqq b
\right)
\rightarrow
\frac{1}{\sqrt{2\pi}}
\int_a^be^{-x^2/2}dx
\quad
(n \rightarrow \infty),$$

which is a typical example of the central limit theorem. 

#### Run the following cell to see the central limit theorem.

In [ ]:
def central_limit(p=0.5,n=1):
    x = np.linspace(-3, 3, 1201)  
    f = np.exp(-x**2/2)/math.sqrt(2*math.pi)
    q=math.floor(10*p)/10

    sigma = np.sqrt(n * p * (1 - p))
    off   = max(100, n)
    z2 = np.zeros(n + 2*off, dtype=float)
    l = np.arange(n + 1, dtype=int)
    logpmf = (
        gammaln(n + 1) - gammaln(l + 1) - gammaln(n - l + 1)
        + l * np.log(p) + (n - l) * np.log1p(-p)
        + np.log(sigma)
    )
    z2[off + l] = np.exp(logpmf)
    z1  = np.zeros(len(x), dtype=float)
    c1  = (len(x) - 1) // 2            
    idx = np.arange(c1)                     
    Lr = np.floor(n*p + (idx/200.0)*sigma).astype(int)
    Ll = np.floor(n*p - (idx/200.0)*sigma).astype(int)    
    right_idx = c1 + np.arange(1, c1 + 1)
    left_idx  = c1 - np.arange(1, c1 + 1)   
    z1[right_idx] = z2[off + 1 + Lr]
    z1[left_idx]  = z2[off + Ll]    
    assert not (np.isnan(z2).any() or np.isnan(z1).any())

    plt.title(f"B(n,p) --> N(0,1), p={q}, n={n}")
    plt.plot(x, f, label="N(0,1)")
    plt.fill_between(x, z1, step="mid", alpha=0.5, color="C1")
    plt.step(x, z1, where='post', label="B(n,p)")
    plt.xlim(-3, 3)
    plt.ylim(0, 0.45)
    plt.xticks([-2, -1, 0, 1, 2])
    plt.yticks([0,1/math.sqrt(2*math.pi)],[0,"1/√2π"])
    plt.grid(False)
    plt.legend()
    plt.show()

interact(central_limit, p=(0.1, 0.9, 0.1), n=(1, 2400));

## 8. Taylor series of $\sin{x}$

It is well-known that $\sin{x}$ ($x\in\mathbb{R}$) equals a power series of $s$: 

$$\sin{x}=\sum_{k=0}^\infty\frac{(-1)^kx^{2k+1}}{(2k+1)!}, \quad x\in\mathbb{R}.$$

The right hand side is said to be the Taylor series of $\sin{x}$ or the Taylor expansion of $\sin{x}$. More precisely, for any $R>0$, 

$$\max_{x\in[-R,R]}
\left\lvert
\sin{x}
-
\sum_{k=0}^K
\frac{(-1)^kx^{2k+1}}{(2k+1)!}
\right\rvert
\rightarrow 0 \quad (K\rightarrow\infty).$$

We now observe this convergence. Set 

$$S_K(x)
:=
\sum_{k=0}^K
\frac{(-1)^kx^{2k+1}}{(2k+1)!}.$$

#### Run the following cell to see this convergence.

In [ ]:
def taylor_sin(K=1):
    x = np.linspace(-3*np.pi, 3*np.pi, num=101)
    y1 = np.sin(x)
    y2 = sum(((-1)**k) * (x**(2*k+1)) / math.factorial(2*k+1) for k in range(K))

    fig, ax = plt.subplots(figsize=(6, 4))

    ax.set_title(f"sin(x) and S_K(x), K={K}")
    ax.plot(x, y1, label="sin(x)")
    ax.plot(x, y2, label=f"S_K(x)")

    ax.set_ylim(-1.2, 1.2)
    ax.set_xlabel("x")

    ticks = [-3*np.pi, -2*np.pi, -np.pi, 0, np.pi, 2*np.pi, 3*np.pi]
    labels = [r"$-3\pi$", r"$-2\pi$", r"$-\pi$", "0", r"$\pi$", r"$2\pi$", r"$3\pi$"]
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels)

    ax.grid(False)
    ax.legend()

    plt.show()

interact(taylor_sin, K=(1, 14, 1));


## 9. Fourier series of a step function and a sawtooth function

Let $g(x)$ and $h(x)$ be a $1$-periodic step function and a $1$ periodic sawtooth function, that is, 

$$\begin{aligned}
  g(x)
& :=
  \begin{cases}
  1, &\ x\in[0,1/2],
  \\
  0, &\ x\in(1/2,1),
  \end{cases}
  \qquad
  g(x):=g(x-n),\ 
  x\in[n,n+1),\  
  n\in\mathbb{Z},
\\
  h(x)
& :=x,
  \qquad
  h(x):=h(x-n),\ 
  x\in[n,n+1),\ 
  n\in\mathbb{Z}.
\end{aligned}$$

Their Fourier series are 

$$\begin{aligned}
  g(x)
& \sim
  \frac{1}{2}
  +
  \sum_{k=1}^\infty
  \frac{2}{(2k-1)\pi}
  \sin\bigl(2(2k-1)\pi x\bigr),
\\
  h(x)
& \sim
  \frac{1}{2}
  -
  \sum_{n=1}^\infty
  \frac{1}{n\pi}
  \sin(2\pi nx),
\end{aligned}$$

Set 

$$\begin{aligned}
  S_{2K-1}[g](x)
& =
  \frac{1}{2}
  +
  \sum_{k=1}^K
  \frac{2}{(2k-1)\pi}
  \sin\bigl(2(2k-1)\pi x\bigr)
  \quad 
  (K=1,2,3.\dotsc),
\\
  S_{-1}[g](x)
& :=\frac{1}{2},
\\
  S_N[h](x)
& =
  \frac{1}{2}
  -
  \sum_{n=1}^N
  \frac{1}{n\pi}
  \sin(2\pi nx)
  \qquad
  (N=1,2,3,\dotsc),
\\
  S_0[h](x)
& :=
  \frac{1}{2}.
\end{aligned}$$

On one hand, at the discontinuous points 

\begin{aligned}
  S_{2K-1}[g](k)
  =
  \frac{1}{2}
& \rightarrow
  \frac{1}{2}
  =
  \frac{g(k-0)+g(k+0)}{2}
  \quad
  (K\rightarrow\infty),
\\
  S_{2K-1}[g](k+1/2)
  =
  \frac{1}{2}
& \rightarrow
  \frac{1}{2}
  =
  \frac{g(k+1/2-0)+g(k+1/2+0)}{2}
  \quad
  (K\rightarrow\infty),
\\
  S_N[h](k)
  =
  \frac{1}{2}
& \rightarrow
  \frac{1}{2}
  =
  \frac{h(k-0)+h(k+0)}{2}
  \quad
  (N\rightarrow\infty)
\end{aligned}

for all $k\in\mathbb{Z}$. On the other hand, we can prove that for any small $\delta>0$

\begin{aligned}
  \max_{x\in[\delta,1/2-\delta]\cup[1/2+\delta,1-\delta]}
  \lvert{S_{2K-1}[g](x)-g(x)}\rvert
& \rightarrow
  0
  \quad
  (K\rightarrow\infty),
\\
  \max_{x\in[\delta,1-\delta]}
  \lvert{S_N[h](x)-h(x)}\rvert
& \rightarrow
  0
  \quad
  (N\rightarrow\infty).
\end{aligned}

The partial sums oscillate violately as $N$ increases near the discontinuous points. This is called the Gibbs phenomenon.

#### Run the following cell to create $g(x)$, $h(x)$, $S_{2K-1}[g](x)$ and $S_N[h](x)$.

In [ ]:
# Set Maximun number of terms M
M = 51

x = np.linspace(-0.1, 1.1, num=481)

# g
g = np.zeros(481)
for i in range(41, 241):
    g[i] = 1
for i in range(441, 481):
    g[i] = 1

# S_{2K-1}[g]
Sg = np.ones((M+1, 481)) / 2
for k in range(1, M):
    for l in range(0, 480):
        Sg[k, l] = Sg[k-1, l] + 2*np.sin(2*(2*k-1)*np.pi*x[l]) / ((2*k-1)*np.pi)
    Sg[k, 480] = Sg[k, 80]

# h
h = x - np.floor(x)

# S_N[h]
Sh = np.ones((M+1, 481)) / 2
for k in range(1, M):
    for l in range(0, 480):
        Sh[k, l] = Sh[k-1, l] - np.sin(2*k*np.pi*x[l]) / (k*np.pi)
    Sh[k, 480] = Sh[k, 80]

def step(K=0):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
    plt.subplots_adjust(wspace=0.5)

    for ax in axes:
        ax.set_xlim(-0.1, 1.1)
        ax.set_ylim(-0.2, 1.2)
        ax.set_xticks([0, 0.5, 1.0])
        ax.set_yticks([0, 0.5, 1.0])
        ax.grid(False)

    axes[0].set_title(f"Step function and its Fourier series, K={K}")
    axes[0].plot(x, Sg[K, :], label=r"$S_{2K-1}[g]$")
    axes[0].plot(x, g, label="g")
    axes[0].set_xlabel("x")
    axes[0].legend(loc=(0.56,0.8))

    axes[1].set_title(f"Sawtooth and its Fourier series, N={K}")
    axes[1].plot(x, Sh[K, :], label=r"$S_N[h]$")
    axes[1].plot(x, h, label="h")
    axes[1].set_xlabel("x")
    axes[1].legend(loc='upper center')

    plt.tight_layout()
    plt.show()

interact(step, K=(0, M-1, 1));


## 10. Weyl's equidistribution theorem

Let $\lfloor{x}\rfloor$ be the floor function for $x\in\mathbb{R}$, that is, 
$$\lfloor{x}\rfloor:=\max\{m \in \mathbb{Z} : m \leqq x\},$$ 
and set 

$$\langle{x}\rangle:=x-\lfloor{x}\rfloor.$$

Then $0\leqq\langle{x}\rangle<1$ and $x\equiv\langle{x}\rangle$ mod $1$.

We say that a sequence $\{a_n\}_{n=1}^\infty \subset [0,1)$ is equidistributed in $[0,1)$ if 

$$\frac{\sharp\{n=1,\dotsc,N : a<a_n<b\}}{N} \rightarrow b-a \quad (N \rightarrow \infty)$$

**Weyl's equidistribution theorem**: For any $\gamma\in\mathbb{R}\setminus\mathbb{Q}$, $\{\langle{n\gamma}\rangle\}_{n=1}^\infty$ is equidistributed in $[0,1)$.

We observe Weyl's equidistribution theorem using a bijection 

$$[0,1)\ni\langle{x}\rangle 
\mapsto 
e^{2\pi{i}\langle{x}\rangle}
=
e^{2\pi{i}x} 
=
\cos(2\pi{x})+i\sin(2\pi{x})
\in \mathbb{S}^1:=\{z\in\mathbb{C} : \lvert{z}\rvert=1\}.$$

#### Run the following cell to see Weyl's equidistribution theorem.

In [ ]:
# Length of seqences
MM=250

tt=np.linspace(0, 1, num=101)
c1=np.cos(2*np.pi*tt)
c2=np.sin(2*np.pi*tt)

def weyl(N=1,gg=2):
    gamma=np.sqrt(gg)
    weylx=np.zeros(MM)
    weyly=np.zeros(MM)

    for n in range(MM):
	    weylx[n]=1.01*np.cos(2*np.pi*gamma*(n+1))
	    weyly[n]=1.01*np.sin(2*np.pi*gamma*(n+1))

    plt.figure(figsize=(5, 5))
    plt.title("Weyl's equidistribution theorem")
    plt.plot(c1, c2, color="magenta", linewidth=1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.text(-0.3, 0.3, f"γ=√{gg}", fontsize=24, color='blue')
    plt.text(-0.5, 0, r"$\exp(2\pi i \gamma n)$", fontsize=24, color='blue')
    plt.text(-0.5, -0.3, f"n=1,...,{N}", fontsize=24, color='blue')
    plt.scatter(weylx[:N], weyly[0:N], color="blue", s=10)
    plt.show()
    
interact(
    weyl,
    N=widgets.IntSlider(min=1, max=MM, step=1, value=1, description="N"),
    gg=widgets.IntSlider(min=2, max=15, step=1, value=2.0, description="γ²")
);